In [5]:
import os
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

import tensorflow as tf
print(tf.__version__)


2.16.1


In [6]:
import os
import cv2
import numpy as np
import tensorflow as tf

# Define the mapping from folder name to label
LABEL_MAPPING = {
    "Tennote": 0,
    "Twentynote": 1,
    "Fiftynote": 2,         # Updated to match your folder name "Fiftynote"
    "1Hundrednote": 3,
    "2Hundrednote": 4,
    "5Hundrednote": 5,
    "2Thousandnote": 6
}

def preprocess_image(file_path, augment=False, target_size=(224, 224)):
    """
    Reads an image from disk, resizes it, and optionally applies augmentation.
    Expects file_path as a string.
    """
    # Ensure file_path is a string (decode if needed)
    if isinstance(file_path, bytes):
        file_path = file_path.decode("utf-8")
    
    img = cv2.imread(file_path)
    if img is None:
        raise ValueError(f"Could not read image: {file_path}")
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    if augment and np.random.rand() > 0.5:
        img = cv2.flip(img, 1)
    return img.astype(np.float32) / 255.0

def parse_image_label(file_path, augment=False):
    """
    Converts a file_path tensor to a Python string, extracts the label from the parent folder,
    preprocesses the image, and returns the image and one-hot encoded label.
    """
    # Convert Tensor to numpy and then to a string
    file_path_np = file_path.numpy()
    if isinstance(file_path_np, bytes):
        file_path_str = file_path_np.decode("utf-8")
    elif isinstance(file_path_np, np.ndarray):
        # If array, take the first element and decode if necessary
        if isinstance(file_path_np[0], bytes):
            file_path_str = file_path_np[0].decode("utf-8")
        else:
            file_path_str = str(file_path_np[0])
    else:
        file_path_str = str(file_path_np)
    
    # Extract folder name to determine label
    folder_name = os.path.basename(os.path.dirname(file_path_str))
    label_int = LABEL_MAPPING.get(folder_name, -1)
    if label_int == -1:
        raise ValueError(f"Unknown folder name: {folder_name}")
    
    # ***Important change:*** Call preprocess_image with file_path_str (which is a plain Python string)
    img = preprocess_image(file_path_str, augment=augment)
    
    # One-hot encode the label for 7 classes:
    one_hot = np.zeros(7, dtype=np.float32)
    one_hot[label_int] = 1.0
    return img, one_hot

def tf_parse_function(file_path, augment=False):
    """
    Wraps parse_image_label for use in a TensorFlow data pipeline.
    """
    img, label = tf.py_function(
        func=lambda fp: parse_image_label(fp, augment),
        inp=[file_path],
        Tout=[tf.float32, tf.float32]
    )
    img.set_shape([224, 224, 3])
    label.set_shape([7])
    return img, label

def get_dataset(data_dir, batch_size=32, augment=False, shuffle=True):
    """
    Creates a tf.data.Dataset from a directory structure where subfolder names represent labels.
    """
    pattern = os.path.join(data_dir, "*", "*.jpg")
    dataset = tf.data.Dataset.list_files(pattern, shuffle=shuffle)
    dataset = dataset.map(lambda fp: tf_parse_function(fp, augment),
                          num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset
